<a href="https://colab.research.google.com/github/capgreen02/zepeto1/blob/main/zepeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/naverz/zepeto-studio-kor/discussions

In [ ]:
## 기본 캐릭터 이동.
import { ZepetoScriptBehaviour } from 'ZEPETO.Script'
import { SpawnInfo, ZepetoPlayers, LocalPlayer, ZepetoCharacter } from 'ZEPETO.Character.Controller'
import { WorldService } from 'ZEPETO.World';
export default class CharacterController extends ZepetoScriptBehaviour {
    Start() {
        //Gets the UserId (logged in through Editor)
        ZepetoPlayers.instance.CreatePlayerWithUserId(WorldService.userId,new SpawnInfo(), true);
        ZepetoPlayers.instance.OnAddedLocalPlayer.AddListener(() => {
            let _player : LocalPlayer = ZepetoPlayers.instance.LocalPlayer;
        });
    }
}     

In [ ]:
##오브젝트 움직임
import { ZepetoScriptBehaviour } from 'ZEPETO.Script';
import { Vector3, GameObject, Object, Time } from "UnityEngine";

export default class Movement extends ZepetoScriptBehaviour 
{
    public movingTime: number = 1;
    public movingRangeX: number = 0;
    public movingRangeY: number = 2;
    private delta: number  = 0;
    private directionFlag: bool = true;
    
    Update() 
    {
        this.delta += Time.deltaTime;
        if(this.delta > this.movingTime) 
        {
            this.delta = 0;
            this.directionFlag = !this.directionFlag;
        }
        
        if (this.directionFlag) 
        {
            //Move upward during the set moving time
            this.transform.Translate(this.movingRangeX * Time.deltaTime, this.movingRangeY * Time.deltaTime, 0);
        }
        else 
        {
            //Move downward during the set moving time
            this.transform.Translate(-1 * this.movingRangeX * Time.deltaTime, -1 * this.movingRangeY * Time.deltaTime, 0);
        }
    }
}



In [ ]:
##텔레포트
import { Collider,Vector3,Quaternion } from 'UnityEngine';
import { ZepetoCharacter, ZepetoPlayer, ZepetoPlayers } from 'ZEPETO.Character.Controller';
import { ZepetoScriptBehaviour } from 'ZEPETO.Script';

export default class Teleport extends ZepetoScriptBehaviour 
{
    
    private zepetoCharacter: ZepetoCharacter;
    
    Start() 
    {    
        //Zepeto character object
        ZepetoPlayers.instance.OnAddedLocalPlayer.AddListener(() => {
            this.zepetoCharacter = ZepetoPlayers.instance.LocalPlayer.zepetoPlayer.character;
        });      
    }

    OnTriggerEnter(collider: Collider) 
    {
        
        if (this.zepetoCharacter == null || collider.gameObject != this.zepetoCharacter.gameObject) 
            return;
        
        //Teleport to Origin Position
        this.zepetoCharacter.Teleport(new Vector3(0, 0, 0), Quaternion.identity);
    }
}

In [ ]:
##아이템 획득
import { Collider, Vector3, GameObject, Transform } from 'UnityEngine';
import { ZepetoCharacter, ZepetoPlayer, ZepetoPlayers } from 'ZEPETO.Character.Controller';
import { ZepetoScriptBehaviour, ZepetoScriptInstance } from 'ZEPETO.Script';
 
// import custom script from path
import UIController from './UIController';
 
export default class GetItem extends ZepetoScriptBehaviour 
{
   public extComponentGameObject: GameObject;
   private extComponent: UIController;
   private zepetoCharacter: ZepetoCharacter;

    Start() 
    {
        //Zepeto character object
        ZepetoPlayers.instance.OnAddedLocalPlayer.AddListener(() => {
            this.zepetoCharacter = ZepetoPlayers.instance.LocalPlayer.zepetoPlayer.character;
        });
        //script import
        this.extComponent = this.extComponentGameObject.GetComponent<UIController>();
    }


    OnTriggerEnter(collider: Collider) 
    {
        if (this.zepetoCharacter == null || collider.gameObject != this.zepetoCharacter.gameObject)
            return;
        
        //Sends a message of raising a count after eating an item
        this.extComponent.IncreaseCount();
        //Destroy item
        GameObject.Destroy(this.gameObject);
   }
}


In [ ]:
##플레이어와 닿을시 점프력 향상
import { Collider } from 'UnityEngine';
import { ZepetoPlayers, ZepetoCharacter } from 'ZEPETO.Character.Controller';
import { ZepetoScriptBehaviour } from 'ZEPETO.Script';

export default class JumpPower extends ZepetoScriptBehaviour 
{
    public jumpPower: number = 5;
    private zepetoCharacter :ZepetoCharacter;

    Start() 
    {
        //Zepeto character object
        ZepetoPlayers.instance.OnAddedLocalPlayer.AddListener(() => {
            this.zepetoCharacter = ZepetoPlayers.instance.LocalPlayer.zepetoPlayer.character;
        });      
    }

    OnTriggerEnter(collider: Collider) 
    {
        if (this.zepetoCharacter == null || collider.gameObject != this.zepetoCharacter.gameObject)
            return;
        
        //Add Jumppower
        this.zepetoCharacter.additionalJumpPower = this.jumpPower;
    }

    OnTriggerExit(collider: Collider) {
        //Turn the jump power back to normal
        this.zepetoCharacter.additionalJumpPower = 0;
    }
}

In [ ]:
##아래 지나갈 시 속도 향상
import { Collider, GameObject, WaitForSeconds } from 'UnityEngine';
import { ZepetoCharacter, ZepetoPlayers } from 'ZEPETO.Character.Controller';
import { ZepetoScriptBehaviour } from 'ZEPETO.Script';

export default class SpeedUp extends ZepetoScriptBehaviour 
{
    private zepetoCharacter :ZepetoCharacter;
    public runSpeed: number = 4;
    public waitTime: number = 3;

    Start() 
    {
        //Zepeto character object
        ZepetoPlayers.instance.OnAddedLocalPlayer.AddListener(() => {
            this.zepetoCharacter = ZepetoPlayers.instance.LocalPlayer.zepetoPlayer.character;
        });
    }
    
    OnTriggerEnter(collider: Collider) 
    {
        if (this.zepetoCharacter == null || collider.gameObject != this.zepetoCharacter.gameObject)
            return;
        this.zepetoCharacter.additionalRunSpeed = this.runSpeed;
        this.StartCoroutine(this.DoRoutine());
    }

    *DoRoutine() 
    {
        //Wait for waitTime value
        yield new WaitForSeconds(this.waitTime);
        //Turn the run speed back to normal
        this.zepetoCharacter.additionalRunSpeed = 0;
    }
}

In [ ]:
##메시지1
import { ZepetoScriptBehaviour } from 'ZEPETO.Script';
import { Text } from "UnityEngine.UI";
import { GameObject } from 'UnityEngine';

export default class UIController extends ZepetoScriptBehaviour 
{
    public resultUI: Text;
    public maxCount: number = 7;
    
    private stringProperty: string;   
    private message: string;
    private count: number;
    
    Start() 
    {
        //Init value
        this.message = "Items";
        this.count = 0;
    }
    
    public IncreaseCount() 
    {
        this.count++;
        
        if(this.count >= this.maxCount) 
        {
            this.resultUI.text = "Clear!";
        } 
        else 
        {
            this.stringProperty = `${this.message} : ${this.count}`;
            this.resultUI.text = this.stringProperty;
        }
    }
     
}

In [ ]:
##박스에 넣는 것
import { Canvas, AnimationClip, WaitForSeconds, GameObject, Object, Random } from 'UnityEngine';
import { Button } from 'UnityEngine.UI';
import { ZepetoPlayers, ZepetoCharacter } from 'ZEPETO.Character.Controller';
import { ZepetoScriptBehaviour } from 'ZEPETO.Script';

// import custom script from path
import UIController from './UIController';

export default class Interaction extends ZepetoScriptBehaviour 
{
    public openUIGesture: Button;
    public interactionCanvas: Canvas;
    public animationClip: AnimationClip;
    public uiControllerObject: GameObject;
    public badEffectFactory: GameObject;
    public goodEffectFactory: GameObject;
    public gift: GameObject;
    public failureRatio: number = 50;
    
    private uiController: UIController;
    private zepetoCharacter :ZepetoCharacter;

    Start() 
    {    
        // Set EventCamera 
        this.interactionCanvas.worldCamera = ZepetoPlayers.instance.ZepetoCamera.camera;
        // Set character
        ZepetoPlayers.instance.OnAddedLocalPlayer.AddListener(() => {
            this.zepetoCharacter = ZepetoPlayers.instance.LocalPlayer.zepetoPlayer.character;
        });
        // Script import
        this.uiController = this.uiControllerObject.GetComponent<UIController>();
        
        //Button Hide
        this.openUIGesture.gameObject.SetActive(false);
        
        //When Button Click
        this.openUIGesture.onClick.AddListener(()=>{
            this.zepetoCharacter.SetGesture(this.animationClip);
            this.StartCoroutine(this.FirstRoutine());         
        });
    }
    
    OnTriggerEnter(collider)
    {
        this.openUIGesture.gameObject.SetActive(true);
    }
    OnTriggerExit(collider)
    {
        this.openUIGesture.gameObject.SetActive(false);
    }
    
    *FirstRoutine() 
    {
        this.uiController.Loading();
        yield new WaitForSeconds(3);
        this.zepetoCharacter.CancelGesture();
        this.RandomCalculation();
    }

    private RandomCalculation()
    {
        let randomNumber: number;
        randomNumber = Random.Range(0,100);
        if (randomNumber <= this.failureRatio)
        {
            this.Lose();
        }
        else
        {
            this.Win();
        }
        this.StartCoroutine(this.SecondRoutine());
    }

    private Lose()
    {
        this.uiController.Lose();
        //GameObject Create
        var obj = Object.Instantiate(this.badEffectFactory) as GameObject;
        obj.transform.position = this.transform.position;
    }

    private Win()
    {
        this.uiController.Win();
        //GameObject Create
        var obj = Object.Instantiate(this.goodEffectFactory) as GameObject;
        obj.transform.position = this.transform.position;
        var giftobj = Object.Instantiate(this.gift) as GameObject;
        giftobj.transform.position = this.transform.position;
    }

    *SecondRoutine()
    {
        yield new WaitForSeconds(1);
        this.uiController.Init();
        //Destroy box
        GameObject.Destroy(this.gameObject);
    }
}